Assuming that the images from male and female are in no particular order

In [4]:
import pandas as pd
import dlib
from tqdm import tqdm
import os

In [3]:
male_fn = '/home/monash/Desktop/fyp-work/fyp-ma-13/fyp-models/preprocessing/combined_filtered/full/male'
female_fn = '/home/monash/Desktop/fyp-work/fyp-ma-13/fyp-models/preprocessing/combined_filtered/full/female'

In [41]:
def split_into_batches(k, fn):
    """
    Splits dataset into k even batches. Any remaining images are ignored9
    
    """
    folder = os.listdir(fn)
    batch_size = len(folder)//k
    splits = []
    for i in range(k):
        splits.append(folder[i:(i+1)*batch_size])
    print("Folder length: ", len(folder))
    print("Batch size: ", batch_size)
    print("Remaining: ", len(folder)%batch_size)
    return splits

In [44]:
male_splits = split_into_batches(6, male_fn)
print()
female_splits = split_into_batches(6, female_fn)

Folder length:  19354
Batch size:  3225
Remaining:  4

Folder length:  22026
Batch size:  3671
Remaining:  0


In [45]:
"""

"""
for split in male_splits:
    print(split)

['6213.jpg', '51121.jpg', '103767.jpg', '52221.jpg', '67412.jpg', '46479.jpg', '33922.jpg', '22713.jpg', '4742.jpg', '32229.jpg', '21050.jpg', '73149.jpg', '13407.jpg', '84558.jpg', '82140.jpg', '45358.jpg', '22884.jpg', '59524.jpg', '31058.jpg', '101150.jpg', '39050.jpg', '27177.jpg', '64166.jpg', '9593.jpg', '77512.jpg', '43944.jpg', '16619.jpg', '33170.jpg', '10037.jpg', '14795.jpg', '55813.jpg', '55174.jpg', '55079.jpg', '65005.jpg', '58141.jpg', '58788.jpg', '68016.jpg', '54848.jpg', '63106.jpg', '37204.jpg', '11499.jpg', '51628.jpg', '32858.jpg', '105269.jpg', '27831.jpg', '18910.jpg', '58405.jpg', '65327.jpg', '79309.jpg', '23566.jpg', '16539.jpg', '82195.jpg', '40691.jpg', '105761.jpg', '58968.jpg', '27591.jpg', '28746.jpg', '103269.jpg', '104907.jpg', '54488.jpg', '30860.jpg', '59968.jpg', '50432.jpg', '100621.jpg', '81979.jpg', '84139.jpg', '30333.jpg', '102750.jpg', '86362.jpg', '43466.jpg', '43543.jpg', '49678.jpg', '32889.jpg', '62997.jpg', '55636.jpg', '12625.jpg', '76568